![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# Generate Portfolio Weights to Run a LEAN Backtest

This notebook connects to PredictNow, optimizes the portfolio weights for each rebalance, and then saves the rebalancing weights for each month into the Object Store. After you run the cells in this notebook, you can run the algorithm in `main.py`, which uses the portfolio weights from PredictNow in a LEAN backtest.

### Connect to PredictNow

In [1]:
from QuantConnect.PredictNowNET import PredictNowClient
from QuantConnect.PredictNowNET.Models import *
from time import sleep
from datetime import datetime

algorithm_start_date = datetime(2023, 1, 1)
algorithm_end_date = datetime(2024, 1, 1)

qb = QuantBook()
qb.settings.daily_precise_end_time = False
client = PredictNowClient("jared@quantconnect.com", "jared_broad")
client.connected

20250204 15:48:41.873 TRACE:: QuantBook started; Is Python: True


True

### Upload Asset Returns

The returns file needs to have sufficient data to cover the backtest period of the algorithm in `main.py` and the in-sample backtest, which occurs before `algorithm_start_date`.

In [2]:
# Calculate the daily returns of the universe constituents.
tickers = 'TIP,BWX,EEM,VGK,IEF,QQQ,EWJ,GLD,VTI,VNQ,TLT,RWX,SPY,DBC,REM,SCZ'.split(',')
symbols = [qb.add_equity(ticker).symbol for ticker in tickers]
df = qb.history(
    symbols, datetime(2010,1,4), algorithm_end_date + timedelta(90), Resolution.DAILY
).close.unstack(0).dropna()
df.index.set_names(['date'], inplace=True)
# Save the returns data into the Object Store.
df.rename(lambda x: x.value, axis='columns', inplace=True)
returns_file_name = "ETF_return_Test.csv"
returns_file_path = qb.object_store.get_file_path(returns_file_name)
df.pct_change().dropna().to_csv(returns_file_path)
# Upload the returns file to PredictNow.
print(client.upload_returns_file(returns_file_path))
# List the return files you've uploaded.
return_files = client.list_returns_files()
','.join(return_files)

Date info from date with 16 return columns. Index range between 2011-03-25 and 2024-03-29.


'ETF_return.csv,ETF_return_Test.csv'

### Upload Constraints

The constraints must contain a subset of the assets in the returns file. The CPO system only provides portfolio weights for assets that have constraints.

In [3]:
# Create the constraints file.
content = '''ticker,LB,UB
SPY,0,0.5
QQQ,0,0.5
VNQ,0,0.5'''
# Save the constraints file in the Object Store.
constraints_file_name = "ETF_constrain_Test.csv"
qb.object_store.save(constraints_file_name, content)
# Upload the constraints file to PredictNow.
constraint_file_path = qb.object_store.get_file_path(constraints_file_name)
message = client.upload_constraint_file(constraint_file_path)
print(message)
# List the constraint files you've uploaded.
constraint_files = client.list_constraint_files()
','.join(constraint_files)

Constraint file processed for 3 components


'ETF_constrain.csv,ETF_constrain_Test.csv'

### (Optional) Upload Features

In [4]:
pass

### Define the Portfolio Parameters

In [5]:
portfolio_parameters = PortfolioParameters(
    name=f"Demo_Project_{datetime.now().strftime('%Y%m%d')}",
    returns_file=returns_file_name,
    constraint_file=constraints_file_name,
    #feature_file=feature_file_name,
    max_cash=1.0,
    rebalancing_period_unit="month",
    rebalancing_period=1,
    rebalance_on="first",
    training_data_size=3,
    evaluation_metric="sharpe"
)

### Run the In-Sample Backtest

The in-sample period must end before the `set_start_date` in `main.py`. Since our algorithm does monthly rebalancing at the beginning of each month, the `training_start_date` argument should align with the start of the month and the `training_end_date` should be one day before the start date in `main.py`.

In [14]:
in_sample_result = client.run_in_sample_backtest(
    portfolio_parameters,
    training_start_date=datetime(2022, 1, 1),   # Align with start of month
    training_end_date=algorithm_start_date-timedelta(1),
    sampling_proportion=0.3,
    debug="debug"
)
print(in_sample_result)

job submitted for cpo in-sample backtesting.: Id b98e0018-0299-45ed-8ce5-57e7abae51a5


The backtest can take some minutes or few hours to complete.

In [58]:
job = client.get_job_for_id(in_sample_result.id)
print(f'{job.status} : {job.progress.message}')

SUCCESS : In sample backtesting copleted, preparing outputs


### Run the Out-Of-Sample Backtest

The out-of-sample period should match the start and end dates of the algorithm `main.py`. It is important to keep the `training_start_date` parameters have the same format for in-sample and out-of-sample tests. For this example, we are working on a portfolio that takes monthly rebalance on the first market day of the month, so we will keep `training_start_date` to the 1st of the month in the out-of-sample test.

In [30]:
out_of_sample_result = client.run_out_of_sample_backtest(
    portfolio_parameters,
    training_start_date=algorithm_start_date,
    training_end_date=algorithm_end_date,
    debug="debug"
)
print(out_of_sample_result)

job submitted for cpo back-testing.: Id 6b003fc9-7f01-4fb1-a379-71b82967800d


The backtest can take some minutes or few hours to complete.

In [59]:
job = client.get_job_for_id(out_of_sample_result.id)
print(f'{job.status} : {job.progress.message}')

SUCCESS : OOS backtesting completed, preparing outputs


### Get the Backtest Weights

Let's get the portfolio weights from the preceding out-of-sample backtest. These are the weights you will use to run the LEAN algorithm in `main.py`. Save the portfolio weights into the Object Store so that you can load them in the algorithm.

In [60]:
weights_by_date = client.get_backtest_weights(
    portfolio_parameters, 
    training_start_date=algorithm_start_date,
    training_end_date=algorithm_end_date,
    debug= "debug"
)
print(weights_by_date)

# Save the weights into the Object Store.
if weights_by_date:
    path = qb.object_store.get_file_path("ETF_Weights_Test1.csv")
    pd.DataFrame(weights_by_date).to_csv(path)

{datetime.datetime(2023, 1, 4, 0, 0): {'SPY': 0.0, 'QQQ': 0.0, 'VNQ': 0.0}, datetime.datetime(2023, 2, 1, 0, 0): {'SPY': 0.0, 'QQQ': 0.0, 'VNQ': 0.0}, datetime.datetime(2023, 3, 1, 0, 0): {'SPY': 0.2035256222, 'QQQ': 0.4201013456, 'VNQ': 0.2165757234}, datetime.datetime(2023, 4, 1, 0, 0): {'SPY': 0.0, 'QQQ': 0.0, 'VNQ': 0.0}, datetime.datetime(2023, 5, 2, 0, 0): {'SPY': 0.0, 'QQQ': 0.0, 'VNQ': 0.0}, datetime.datetime(2023, 6, 1, 0, 0): {'SPY': 0.0, 'QQQ': 0.0, 'VNQ': 0.0}, datetime.datetime(2023, 7, 1, 0, 0): {'SPY': 0.4703326844, 'QQQ': 0.4703326844, 'VNQ': 0.0}, datetime.datetime(2023, 8, 1, 0, 0): {'SPY': 0.4670399763, 'QQQ': 0.1664494948, 'VNQ': 0.3005904815}, datetime.datetime(2023, 9, 1, 0, 0): {'SPY': 0.4567448111, 'QQQ': 0.4567448111, 'VNQ': 9.46009249e-17}, datetime.datetime(2023, 10, 3, 0, 0): {'SPY': 0.4610234006, 'QQQ': 0.4610234006, 'VNQ': 0.0}, datetime.datetime(2023, 11, 1, 0, 0): {'SPY': 0.0, 'QQQ': 0.0, 'VNQ': 0.0}, datetime.datetime(2023, 12, 1, 0, 0): {'SPY': 0.35096